# Underdog Recordstore Events Parser
This notebook fetches events from [underdogrecordstore.de/vorverkauf](https://underdogrecordstore.de/vorverkauf)
and enriches them with Spotify genre + artist link.

In [19]:
!pip install requests beautifulsoup4 spotipy

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import musicbrainzngs

In [21]:
# Spotify Setup
SPOTIFY_CLIENT_ID = "31a82b3bec19482c8b4f9f9e4730f1dd"
SPOTIFY_CLIENT_SECRET = "4b3a6c5798e8401c9d36784f57558847"

spotify = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET
    )
)

In [22]:
def get_spotify_info(artist_name):
    try:
        result = spotify.search(q=f"artist:{artist_name}", type="artist", limit=1)
        items = result.get("artists", {}).get("items", [])
        if not items:
            return None, None
        artist = items[0]
        genres = artist.get("genres", [])
        link = artist.get("external_urls", {}).get("spotify")
        return genres, link
    except Exception as e:
        print("Spotify error:", e)
        return None, None

In [24]:
# Setup
musicbrainzngs.set_useragent("UnderdogEventsParser", "1.0")

def get_musicbrainz_genre(artist_name):
    try:
        result = musicbrainzngs.search_artists(artist=artist_name, limit=1)
        if result['artist-list']:
            artist = result['artist-list'][0]
            # MusicBrainz has 'tag-list' or 'type'
            tags = [t['name'] for t in artist.get('tag-list', [])]
            return tags
    except Exception as e:
        print("MusicBrainz error:", e)
    return None

In [ ]:
from asyncio import events


URL = "https://underdogrecordstore.de/vorverkauf"

def fetch_events():
    resp = requests.get(URL)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    lines = [t.strip() for t in soup.stripped_strings]

    # Match date (dd.mm.), band, location, price
    event_pattern = re.compile(
        r"""
        ^\s*
        (?P<date>\d{2}\.\d{2}\.)                              # date: DD.MM.
        \s+
        (?P<band>.+?)                                         # band (lazy)
        (?=                                                   # band boundary lookahead
            (?:\s+@\s|                                        # next is '@ location'
            \s+[*]?(?:ab\s*)?\d+(?:[.,]\d{1,2})?\s*€?      # or price
            |\s*(?:$|\(|Ausverkauft|verlegt)               # or end/paren/status
            )
        )
        (?:\s+@\s*(?P<location>.+?)                           # optional @ location
            (?=(?:\s+[*]?(?:ab\s*)?\d+(?:[.,]\d{1,2})?\s*€?   # stop before price
            |\s*(?:$|\(|Ausverkauft|verlegt))              # or end/paren/status
            )
        )?
        (?:\s+[*]?(?P<price_prefix>ab\s*)?                    # optional * and 'ab '
            (?P<price>\d+(?:[.,]\d{1,2})?)\s*€?               # optional price
        )?
        (?:\s*(?P<status>Ausverkauft!?|verlegt[^\n]*))?       # optional status
        """,
        re.VERBOSE,
    )

    now = datetime.now()
    events = []
    section = ""
        
    for line in lines:
        m = event_pattern.search(line)
        if not m:
            section = line
            continue
        
        date, band, location, price, status = m.group("date", "band", "location", "price", "status")

        if line != "Underdog Shows:":
            location = section[:-1].strip()

        print(f"Parsing line: {band} on {date} at {location} for {price} EUR")

        # Add current year to date
        d = datetime.strptime(date + str(now.year), "%d.%m.%Y")

        # If the parsed date is in the past, assume next year
        if d.date() < now.date():
            d = d.replace(year=now.year + 1)

        # genres, link = get_spotify_info(band)
        # if not genres:
        #     genres = get_musicbrainz_genre(band)
        # time.sleep(0.1)

        band = band or ""
        location = location or ""

        genres, link = "", ""

        events.append({
            "date": d,
            "band": band.strip(),
            "location": location.strip(),
            "price_eur": "" if price is None else float(price.strip().replace(",", ".")),
            "genres": genres or "",
            "spotify_link": link or "",
            "status": status or "",
        })

    # ✅ Sort by true datetime
    # return sorted(events, key=lambda x: x["date"])
    return events


events = fetch_events()
# len(events)

Parsing line: Ritual on 02.10. at  for 37 EUR
Parsing line: Duesenjaeger on 03.10. at  for 9 EUR
Parsing line: Snake Eyes on 15.10. at  for 18 EUR
Parsing line: Beach Bunny on 15.10. at  for 33,50 EUR
Parsing line: ClickClickDecker on 18.10. at  for 27 EUR
Parsing line: Grillmaster Flash on 23.10. at  for 20 EUR
Parsing line: DYSE on 04.11. at  for 9 EUR
Parsing line: Poison Ruin on 06.11. at  for 25 EUR
Parsing line: Erik Cohen on 08.11. at  for 9 EUR
Parsing line: Gorilla Biscuits, Terror, No Pressure on 17.11. at  for 41,50 EUR
Parsing line: Saturdays at your place on 18.01. at  for 37 EUR
Parsing line: A.A. Williams on 26.02. at  for 9 EUR
Parsing line: Kafvka on 13.03. at  for 9 EUR
Parsing line: Masters Of Reality on 02.10. at  for 38,45 EUR
Parsing line: Bulgarian Cartrader on 07.10. at  for 28 EUR
Parsing line: Power Plush on 10.10. at  for 28 EUR
Parsing line: Sports Team on 12.10. at  for 28,15 EUR
Parsing line: ClickClickDecker on 18.10. at  for 27 EUR
Parsing line: Loki on 

In [189]:
URL = "https://underdogrecordstore.de/vorverkauf"

def fetch_locations():
    resp = requests.get(URL)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    lines = [t.strip() for t in soup.stripped_strings]
    return [line[:-1].strip() for line in lines if line.endswith(":")]

def fetch_events():
    locations = fetch_locations()
    # loc_pattern = r"|".join(re.escape(loc) for loc in locations)

    loc_pattern = r"|".join(
        re.escape(loc) for loc in sorted(locations, key=len, reverse=True)
    )

    print("loc_pattern:", loc_pattern)

    # Regex to parse a concert line
    event_pattern = re.compile(
        # r"""
        # ^\s*
        # (?P<date>\d{{2}}\.\d{{2}}\.)                              # date: DD.MM.
        # \s+
        # (?P<band>.+?)                                             # band (lazy)
        # (?=                                                       # band boundary lookahead
        #     (?:\s+@\s|                                            # '@ location'
        #        \s+[*]?(?:ab\s*)?\d+(?:[.,]\d{{1,2}})?\s*€?        # or price
        #      |\s*(?:$|\(|Ausverkauft|verlegt)                     # or end/paren/status
        #     )
        # )
        # (?:\s+@\s*(?P<location>(?:{loc_pattern}|.+?)))?           # optional @ location
        # (?:\s+[*]?(?P<price_prefix>ab\s*)?                        # optional * and 'ab '
        #     (?P<price>\d+(?:[.,]\d{{1,2}})?)\s*€?                 # optional price
        # )?
        # (?:\s*(?P<status>Ausverkauft!?|verlegt[^\n]*))?           # optional status
        
        
        
        # ^\s*
        # (?P<date>\d{2}\.\d{2}\.)                              # date: DD.MM.
        # \s+
        # (?P<band>.+?)                                         # band
        # (?=                                                   # band boundary lookahead
        #     (?:\s+@\s|                                        # '@ location'
        #        \s+[*]?(?:ab\s*)?\d+(?:[.,]\d{1,2})?\s*€?      # or price
        #      |\s*(?:$|\(|Ausverkauft|verlegt)                 # or end/paren/status
        #     )
        # )
        # (?:\s+@\s*(?P<location>(?:{loc_pattern}|.+?))
        #     (?=(?:\s+[*]?(?:ab\s*)?\d+(?:[.,]\d{1,2})?\s*€?   # stop before price
        #        |\s*(?:$|\(|Ausverkauft|verlegt))              # or end/paren/status
        #     )
        # )?       # optional @ location
        # # (?:\s+@\s*(?P<location>.+?)                           # optional @ location
        # #     (?=(?:\s+[*]?(?:ab\s*)?\d+(?:[.,]\d{1,2})?\s*€?   # stop before price
        # #        |\s*(?:$|\(|Ausverkauft|verlegt))              # or end/paren/status
        # #     )
        # # )?
        # (?:\s+[*]?(?P<price_prefix>ab\s*)?                    # optional 'ab' or '*'
        #     (?P<price>\d+(?:[.,]\d{1,2})?)\s*€?               # optional price
        # )?
        # (?:\s*(?P<status>Ausverkauft!?|verlegt[^\n]*))?       # optional status

        
        # ^\s*
        # (?P<date>\d{{2}}\.\d{{2}}\.)                              # Datum: DD.MM.
        # \s+
        # (?P<band>.+?)                                             # Band (lazy)
        # (?=                                                       # Band endet vor:
        #     (?:\s+@\s|                                            #   '@ location'
        #        \s+[*]?(?:ab\s*)?\d+(?:[.,]\d{{1,2}})?\s*€?        #   Preis
        #      |\s*(?:$|\(|Ausverkauft|verlegt)                     #   EOL/ Klammer / Status
        #     )
        # )
        # (?:\s+@\s*(?P<location>(?:{loc_pattern})))?               # optionale @ Location (nur bekannte)
        # (?:\s+[*]?(?P<price_prefix>ab\s*)?                        # optional '*' / 'ab'
        #     (?P<price>\d+(?:[.,]\d{{1,2}})?)\s*€?                 # optional Preis
        # )?
        # (?:\s*(?P<status>Ausverkauft!?|verlegt[^\n]*))?           # optionaler Status


        rf"""
        ^\s*
        (?P<date>\d{{2}}\.\d{{2}}\.)                   # date: DD.MM.
        \s+
        (?P<band>.+?)                                  # band (lazy)
        (?=                                            # band ends before:
            (?:\s+@\s|                                 #   '@ location'
               \s+[*]?(?:ab\s*)?\d+(?:[.,]\d{{1,2}})?  #   price
             |\s*(?:$|\(|Ausverkauft|verlegt)          #   EOL/paren/status
            )
        )
        (?:\s+@\s*(?P<location>(?:{loc_pattern})))?    # @ location
        (?:\s+[*]?(?P<price_prefix>ab\s*)?             # price part (optional)
            (?P<price>\d+(?:[.,]\d{{1,2}})?)\s*€?
        )?
        (?:\s*(?P<status>Ausverkauft!?|verlegt[^\n]*))?  # status (can appear without price)
        """,
        re.VERBOSE | re.UNICODE | re.IGNORECASE,
    )

    resp = requests.get(URL)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    lines = [t.strip() for t in soup.stripped_strings]

    now = datetime.now()
    events = []
    section = None   # current heading

    for line in lines:
        # Section headings (like "Artheater:")
        if line.endswith(":"):
            section = line[:-1].strip()
            continue

        m = event_pattern.search(line)
        if not m:
            continue

        date, band, loc_at, price, status = m.group("date", "band", "location", "price", "status")

        # prefer explicit @location, else section heading
        location = loc_at if loc_at else (section or "")

        # build proper datetime
        d = datetime.strptime(date + str(now.year), "%d.%m.%Y")
        if d.date() < now.date():  # roll over to next year if already past
            d = d.replace(year=now.year + 1)

        # normalize price
        price_eur = float(price.replace(",", ".")) if price else None

        events.append({
            "origin": line,
            "date": d,
            "band": band.strip(),
            "location": location.strip(),
            "price_eur": price_eur,
            "status": status or "",
        })

        print(f"Parsing line: {band.strip()} on {d.strftime('%d.%m.%Y')} at {location} for {price_eur} EUR {status or ''}")

    # sort chronologically
    # return sorted(events, key=lambda x: x["date"])
    return events

# Example usage
events = fetch_events()
print(f"Parsed {len(events)} events")

loc_pattern: Club\ Bahnhof\ Ehrenfeld|Die\ Hängenden\ Gärten|Vinyl\ Reservierungen|Carlswerk\ Victoria|Stereo\ Wonderland|Allgemeine\ Fragen|Wohngemeinschaft|Live\ Music\ Hall|Stadthalle\ Köln|Underdog\ Shows|Sonic\ Ballroom|Bumann\ \&\ Sohn|Lanxess\ Arena|Kulturkirche|Philharmonie|Tsunami\ Club|Essigfabrik|Stadtgarten|Tanzbrunnen|Blue\ Shell|Club\ Volta|Stollwerck|Artheater|Gebäude\ 9|Helios\ 37|Palladium|Südbrücke|Kantine|E\-Werk|Gloria|Subway|Luxor|Jaki|Yuca|MTC|MTC
Parsing line: Ritual on 02.10.2025 at Helios 37 for 22.5 EUR 
Parsing line: Duesenjaeger on 03.10.2025 at Gebäude 9 for 24.5 EUR 
Parsing line: Snake Eyes on 15.10.2025 at Stereo Wonderland for 18.0 EUR 
Parsing line: Beach Bunny on 15.10.2025 at Gloria for 33.5 EUR 
Parsing line: ClickClickDecker on 18.10.2025 at Artheater for 27.0 EUR 
Parsing line: Grillmaster Flash on 23.10.2025 at Stereo Wonderland for 20.0 EUR 
Parsing line: DYSE on 04.11.2025 at Gebäude 9 for 29.5 EUR 
Parsing line: Poison Ruin on 06.11.2025 at Cl

In [157]:
events = fetch_events()

len(events)

events[:5]


Parsing line: Ritual on 02.10.2025 at Helios for 37.0 EUR 
Parsing line: Duesenjaeger on 03.10.2025 at Gebäude for 9.0 EUR 
Parsing line: Snake Eyes on 15.10.2025 at Stereo Wonderland for 18.0 EUR 
Parsing line: Beach Bunny on 15.10.2025 at Gloria for 33.5 EUR 
Parsing line: ClickClickDecker on 18.10.2025 at Artheater for 27.0 EUR 
Parsing line: Grillmaster Flash on 23.10.2025 at Stereo Wonderland for 20.0 EUR 
Parsing line: DYSE on 04.11.2025 at Gebäude for 9.0 EUR 
Parsing line: Poison Ruin on 06.11.2025 at Club Volta for 25.0 EUR 
Parsing line: Erik Cohen on 08.11.2025 at Gebäude for 9.0 EUR 
Parsing line: Gorilla Biscuits, Terror, No Pressure on 17.11.2025 at Kantine for 41.5 EUR 
Parsing line: Saturdays at your place on 18.01.2026 at Helios for 37.0 EUR 
Parsing line: A.A. Williams on 26.02.2026 at Gebäude for 9.0 EUR 
Parsing line: Kafvka on 13.03.2026 at Gebäude for 9.0 EUR 
Parsing line: Masters Of Reality on 02.10.2025 at Artheater for 38.45 EUR 
Parsing line: Bulgarian Cartra

[{'date': datetime.datetime(2025, 9, 13, 0, 0),
  'band': 'Moneybrother',
  'location': 'Gebäude 9',
  'price': 38.2,
  'status': 'Ausverkauft!'},
 {'date': datetime.datetime(2025, 9, 13, 0, 0),
  'band': 'Gogol Bordello',
  'location': 'Live Music Hall',
  'price': 46.95,
  'status': ''},
 {'date': datetime.datetime(2025, 9, 13, 0, 0),
  'band': 'The Slapstickers',
  'location': 'Sonic Ballroom',
  'price': 17.6,
  'status': ''},
 {'date': datetime.datetime(2025, 9, 14, 0, 0),
  'band': 'Greg Freeman',
  'location': 'Blue Shell',
  'price': 29.3,
  'status': ''},
 {'date': datetime.datetime(2025, 9, 14, 0, 0),
  'band': 'Richy Mitch & The Coal Miners',
  'location': 'Gebäude 9',
  'price': 32.75,
  'status': ''}]

In [2]:
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime

URL = "https://underdogrecordstore.de/vorverkauf"

# ---------- utilities ----------

def get_lines_from_page(url: str):
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    text = soup.get_text("\n", strip=True)          # robust split into lines
    return [ln.strip() for ln in text.splitlines() if ln.strip()]

def fetch_locations_from_headings(lines):
    """Section headings (end with ':') become known locations."""
    return [ln[:-1].strip() for ln in lines if ln.endswith(":")]

def build_event_pattern():
    """
    Base event regex that:
    - captures date, band, optional @location, optional price
    - uses lookaheads so band/location stop before price/status/paren/EOL
    - captures the full trailing status text into 'status_text'
    """
    return re.compile(
        r"""
        ^\s*
        (?P<date>\d{2}\.\d{2}\.)\s+                 # DD.MM.
        (?P<band>.+?)                               # band (lazy)
        (?=                                         # band boundary
            (?:\s+@\s|                              # '@ location' ahead
               \s+[*]?(?:ab\s*)?\d+(?:[.,]\d{1,2})?\s*€? |  # or price
               \s*(?:$|\(|Ausverkauft|Verlegt)      # or EOL/paren/status
            )
        )
        (?:\s+@\s*
            (?P<location>.+?)                       # @ location (digits allowed)
            (?=                                     # location boundary
               (?:\s+[*]?(?:ab\s*)?\d+(?:[.,]\d{1,2})?\s*€? |
                  \s*(?:$|\(|Ausverkauft|Verlegt)
               )
            )
        )?
        (?:\s+[*]?(?:ab\s*)?
            (?P<price>\d+(?:[.,]\d{1,2})?)\s*€?     # optional price
        )?
        (?:\s*(?P<status_text>(?:Ausverkauft!?|[Vv]erlegt)[^\n]*))?  # full status text
        """,
        re.VERBOSE | re.UNICODE
    )

# status helpers
ARTICLES = r"(?:die|den|das|dem|der)\s+"
PREPS    = r"(?:in|nach|vom|von)\s+"      # most common forms

STATUS_PATTERNS = [
    (re.compile(r"\bausverkauft!?$", re.I), "ausverkauft"),
    (re.compile(r"\babgesagt!?$", re.I), "abgesagt"),
    (re.compile(r"\bverlegt\b.*", re.I), "verlegt"),
]

def parse_status(line: str, known_locs):
    """
    Extract (status_kind, new_location, status_raw) from the line.
    """
    status_kind, new_location, status_raw = "", "", ""

    for pat, kind in STATUS_PATTERNS:
        m = pat.search(line)
        if not m:
            continue
        status_kind = kind
        status_raw = m.group(0).strip()
        break

    # if verlegt → try to detect target location
    if status_kind == "verlegt":
        s = status_raw
        for loc in sorted(known_locs, key=len, reverse=True):
            if re.search(rf"\b{re.escape(loc)}\b", s, flags=re.I):
                new_location = loc
                break
        if not new_location:
            mm = re.search(r"(?:in|nach|vom|von\s+(?:die|den|das|dem|der))?\s*(?P<loc>[^,(]+)", s, re.I)
            if mm:
                new_location = mm.group("loc").strip()

    return status_kind, new_location, status_raw

# ---------- main ----------

def fetch_events():
    lines = get_lines_from_page(URL)
    known_locations = fetch_locations_from_headings(lines)
    event_re = build_event_pattern()

    now = datetime.now()
    events = []
    section = None

    for line in lines:
        if line.endswith(":"):
            section = line[:-1].strip()
            continue

        m = event_re.match(line)
        if not m:
            continue

        gd = m.groupdict()
        date = gd["date"]
        band = (gd["band"] or "").strip()
        at_loc = (gd.get("location") or "").strip()
        price_raw = gd.get("price")
        status_text = gd.get("status_text") or ""

        # prefer explicit @location; else section heading
        location = at_loc if at_loc else (section or "")

        # date → with year; roll over if already past this year
        d = datetime.strptime(date + str(now.year), "%d.%m.%Y")
        if d.date() < now.date():
            d = d.replace(year=now.year + 1)

        # price normalize
        price_eur = float(price_raw.replace(",", ".")) if price_raw else None

        # structured status
        status_kind, new_location, status_raw = parse_status(line, known_locations)

        events.append({
            "origin": line,
            "date": d,
            "band": band,
            "location": location,
            "price_eur": price_eur,
            "status_kind": status_kind,     # 'ausverkauft' | 'verlegt' | 'abgesagt' | ''
            "new_location": new_location,   # Ziel-Location falls 'verlegt'
            "status_raw": status_raw,       # original status string
            "section": section,
        })

    events.sort(key=lambda x: x["date"])
    return events

events = fetch_events()

In [16]:
import pandas as pd
from datetime import datetime

def save_events_csv(events, basepath="events"):
    """
    Save events to a timestamped JSON file.
    - basepath: without extension, e.g. 'events' → events_YYYYMMDD_HHMMSS.csv
    """
    df = pd.DataFrame(events)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    json_path = f"{basepath}_{ts}.json"

    # add id column starting from 1
    df.insert(0, "id", range(1, len(df) + 1))

    # pretty print to JSON, no records orientation
    df.to_json(json_path, index=False, indent=4, date_format="iso", force_ascii=False, orient="records")

    print(f"✅ Saved {len(df)} events to {json_path}")
    return json_path

json_file = save_events_csv(events, basepath="concert_events")

✅ Saved 374 events to concert_events_20250919_005906.json


In [15]:
import os
import glob
import pandas as pd

def load_latest_events(basepath="concert_events"):
    """
    Find the latest JSON file saved by save_events_csv() and return a list of dicts.
    """
    pattern = f"{basepath}_*.json"
    files = glob.glob(pattern)
    if not files:
        raise FileNotFoundError(f"No files found matching {pattern}")

    latest_file = max(files, key=os.path.getmtime)

    df = pd.read_json(latest_file)
    # Replace NaN with None
    #df = df.where(pd.notnull(df), None)
    
    events = df.to_dict(orient="records")

    print(f"✅ Loaded {len(events)} events from {latest_file}")
    return events

loaded_events = load_latest_events("concert_events")


✅ Loaded 374 events from concert_events_20250919_005633.json


In [11]:
from IPython.display import HTML

for e in loaded_events:
    print(f"\n")
    print(f"Origin: {e['origin']}")
    print(f"{e['date'].date()} — {e['band']} — {e['location']} — {e['price_eur']} €")
    print(f"   Status: {e['status_kind']} {e['new_location']}")
    print(f"   Raw: {e['status_raw']}")
    # print(f"   Genres: {e['genres']}")
    # print(f"   Spotify: {e['spotify_link']}")



Origin: 19.09. Poison The Well 46,95 € Ausverkauft!
2025-09-19 — Poison The Well — Gebäude 9 — 46.95 €
   Status: ausverkauft 
   Raw: Ausverkauft!


Origin: 19.09. Popperklopper *14,30 €
2025-09-19 — Popperklopper — Sonic Ballroom — 14.3 €
   Status:  
   Raw: 


Origin: 19.09. Donots 52,75 €
2025-09-19 — Donots — Südbrücke — 52.75 €
   Status:  
   Raw: 


Origin: 20.09. Müllem Mon Amour 46,20 € (Hardtickets) Abgesagt!
2025-09-20 — Müllem Mon Amour — Carlswerk Victoria — 46.2 €
   Status: abgesagt 
   Raw: Abgesagt!


Origin: 20.09. Young Rebel Set 37,20
2025-09-20 — Young Rebel Set — Gebäude 9 — 37.2 €
   Status:  
   Raw: 


Origin: 20.09. Mogwai 50,32 €
2025-09-20 — Mogwai — Live Music Hall — 50.32 €
   Status:  
   Raw: 


Origin: 20.09. The Vovos *15,40 €
2025-09-20 — The Vovos — Sonic Ballroom — 15.4 €
   Status:  
   Raw: 


Origin: 20.09. Antilopden Gang 53,05 €
2025-09-20 — Antilopden Gang — Südbrücke — 53.05 €
   Status:  
   Raw: 


Origin: 21.09. Nova Twins 28,15 €
2025

In [162]:
from IPython.display import HTML

print(f"Events: {len(events)}")

def render_events(events):
    html = "<ul style='list-style:none; padding:0;'>"
    for e in events:
        weekday = e['date'].strftime("%a")
        html += f"""
        <li style="margin:10px 0; padding:10px; border:1px solid #ddd; border-radius:8px;">
            <b>{weekday}. {e['date'].date()} — {e['band']} — {e['location']} — {e['price_eur']} €</b><br>
            <i>Genres:</i> {', '.join(e['genres']) if isinstance(e['genres'], (list, tuple)) else e['genres']}<br>
            <a href="{e['spotify_link']}" target="_blank" style="color:#1DB954; text-decoration:none;">🎵 Open in Spotify</a>
        </li>
        """
    html += "</ul>"
    return HTML(html)

render_events(events)

Events: 374


KeyError: 'genres'